Транзакция биткоина представляет собой набор входных и выходных адресов, а также указание сколько каждый адрес вносит биткоинов в данную транзакцию (для входных) или забирает из транзакции (для выходных адресов). Стоит заметить, что ничего не известно о том, сколько биткоинов переправляет конкретный входной адрес на любой другой из выходных. Также часть средств из суммы входов уходит в качестве сбора (fee) для майнеров при включении транзакции в новый блок. Мы можем найти fee (fee >= 0) как разницу между суммой входов и суммой выходов транзакции.

-- Описание структуры файлов

address_clust.csv включает в себя набор адресов из 1 и 2 кластера с указанием данного кластера.
address_id - адрес
cluster_id - кластер

address_stats.csv включает в себя описание всех транзакций с участием в них адресов из файла address_clust.csv.
id - просто id записи
address_id - адрес
transaction_id - транзакция
received - сумма в сатоши (1 биткоин = 10^8 сатоши) полученная данным адресом в данной транзакции
sent - сумма в сатоши отправленная данным адресом в данной транзакции

Помимо имеющихся в address_clust.csv адресов, файл address_stats.csv содержит еще дополнительные адреса, являющиеся внешними по отношению к кластеру 1 и 2, можете сопоставить им всем кластер 0.

-- Задача

Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?
Доп. вопросы:
Сколько биткоинов отправил кластер 1 в кластер 0 и сколько кластер 0 отправил в кластер 1?
Сколько биткоинов отправил кластер 2 в кластер 0 и сколько кластер 0 отправил в кластер 2?
Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

In [1]:
import pandas as pd

In [2]:
data_clust  = pd.read_csv('address_clust.csv')
data_stats = pd.read_csv('address_stats.csv')

In [3]:
data_clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [4]:
data_stats.head(5)

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [5]:
data_stats.shape

(76724, 5)

In [6]:
#уберем дубликаты записей
data_stats.drop_duplicates(inplace=True)

In [7]:
data_stats.shape

(54445, 5)

In [8]:
#объеденим два сета по столбцу address_id
data = data_stats.merge(data_clust, how='left', on='address_id')

In [9]:
data.head(5)

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,NaN
1,87134766,45919235,19162323,0.0,1071052.0,NaN
2,87134767,46529090,19162323,1000000.0,0.0,1.0
3,87134768,46529091,19162323,1821052.0,0.0,NaN
4,87154706,46529090,19166856,0.0,1000000.0,1.0


In [10]:
#заменим NaN значения в столбце cluster_id на 0
data.cluster_id = data.cluster_id.fillna(0)

In [11]:
#создаем сет только для 1 кластера
data_1 = data[data.cluster_id == 1]
#создаем сет для 2 кластера
data_2 = data[data.cluster_id == 2]
#создаем сет для 0 кластера
data_0 = data[data.cluster_id == 0]

Сколько биткоинов отправлено из 1 кластера во 2, сколько биткоинов отправлено из 2 кластера в 1 кластер?

Для нахождения суммы биткоинов отправленных из первого кластера во второй, мы пройдемся по уникальному списку всех транзакций где spent > 0 в 1 кластере, просуммируем, и вычтем received этой же транзакции 1 кластера, вычтем received этой же транзакции 0 кластера                                                                 

In [31]:
send_1_to_2_btс = 0
for i in data_1[data_1.sent > 0].transaction_id.unique():
    send_1_to_2_btс += (data_1[data_1.transaction_id == i]['sent'].sum() 
                        - data_1[data_1.transaction_id == i]['received'].sum() 
                        - data_0[data_0.transaction_id == i]['received'].sum())
    
print('{} Биткоинов отправлено из 1 кластера вo 2 кластер'.format(send_1_to_2_btс/10**8)) 

65.123964 Биткоинов отправлено из 1 кластера вo 2 кластер


Аналогично проделываем для нахождения суммы биткоинов из 2 кластера в 1

In [32]:
send_2_to_1_btс = 0
for i in data_2[data_2.sent > 0].transaction_id.unique():
    send_2_to_1_btс += (data_2[data_2.transaction_id == i]['sent'].sum() 
    - data_2[data_2.transaction_id == i]['received'].sum() 
    - data_0[data_0.transaction_id == i]['received'].sum())
    
print('{} Биткоинов отправлено из 2 кластера в 1 кластер'.format(send_2_to_1_btс/10**8))

68.28295861 Биткоинов отправлено из 2 кластера в 1 кластер


Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

In [20]:
fee_1 = 0
for i in data_1[data_1.sent > 0].transaction_id.unique():
    fee_1 += (data[data.transaction_id == i].sent.sum() 
              - data[data.transaction_id == i].received.sum())
    
print('{} Биткоинов на fee в 1 кластере потрачено'.format(fee_1/10**8))

0.502344 Биткоинов на fee в 1 кластере потрачено


In [21]:
fee_2 = 0
for i in data_2[data_2.sent > 0].transaction_id.unique():
    fee_2 += (data[data.transaction_id == i].sent.sum() 
              - data[data.transaction_id == i].received.sum())
    
print('{} Биткоинов на fee в 2 кластере потрачено'.format(fee_2/10**8))

0.37295861 Биткоинов на fee в 2 кластере потрачено


Сколько биткоинов отправлено из 0 кластера в 1, сколько биткоинов отправлено из 1 кластера в 0 кластер?

Считать будем аналогично, только для правильного результата из общей суммы надо вычесть fee  кластера, т.к. мы не учитываем это в формуле

In [59]:
send_1_to_0_btс = 0
for i in data_1[data_1.sent > 0].transaction_id.unique():
    send_1_to_0_btс += (data_1[data_1.transaction_id == i]['sent'].sum()
                        - data_1[data_1.transaction_id == i]['received'].sum()
                        - data_2[data_2.transaction_id == i]['received'].sum()) 
    
print('{} Биткоинов отправлено из 1 кластера в 0 кластер'.format((send_1_to_0_btс/10**8) - fee_1/10**8))

291.5108346 Биткоинов отправлено из 1 кластера в 0 кластер


In [17]:
send_0_to_1_btс = 0
for i in data_0[data_0.sent > 0].transaction_id.unique():
    send_0_to_1_btс += (data_0[data_0.transaction_id == i]['sent'].sum()
                        - data_0[data_0.transaction_id == i]['received'].sum() 
                        - data_2[data_2.transaction_id == i]['received'].sum())

print('{} Биткоинов отправлено из 0 кластера в 1 кластер'.format(send_0_to_1_btс/10**8))

289.67679244 Биткоинов отправлено из 0 кластера в 1 кластер


Сколько биткоинов отправлено из 0 кластера во 2, сколько биткоинов отправлено из 2 кластера в 0 кластер?

In [62]:
send_2_to_0_btс = 0
for i in data_2[data_2.sent > 0].transaction_id.unique():
    send_2_to_0_btс += (data_2[data_2.transaction_id == i]['sent'].sum() 
                        - data_2[data_2.transaction_id == i]['received'].sum() 
                        - data_1[data_1.transaction_id == i]['received'].sum())
    
print('{} Биткоинов отправлено из 2 кластера в 0 кластер'.format(send_2_to_0_btс/10**8 - fee_2/10**8))

2.43507 Биткоинов отправлено из 2 кластера в 0 кластер


In [19]:
send_0_to_2_btс = 0
for i in data_0[data_0.sent > 0].transaction_id.unique():
    send_0_to_2_btс += (data_0[data_0.transaction_id == i]['sent'].sum() 
                        - data_0[data_0.transaction_id == i]['received'].sum() 
                        - data_1[data_1.transaction_id == i]['received'].sum()) 

print('{} Биткоинов отправлено из 0 кластера в 2 кластер'.format(send_0_to_2_btс/10**8))

6.98261245 Биткоинов отправлено из 0 кластера в 2 кластер
